#### **<font color='green'>Import Lib</font>**

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

<font color='green'>**Import data**</font>

In [48]:
df = pd.read_csv('data.csv')
df = df[["Date", "RF_LeThuy"]]

#### **Pro-precessing**

In [49]:
# Replace 'x' = mode_value
mode_value = df.iloc[:, 1].mode()[0]
df.iloc[:, 1] = df.iloc[:, 1].replace('x', mode_value)
df['Month'] = df['Date'].str.split('/').str.get(0)


In [50]:
# Del col Date
df = df.drop('Date', axis=1)

In [51]:
df = df.apply(pd.to_numeric, errors='coerce')
# df = df.astype(float)

In [52]:
def levelOfRainfall(x):
  if (x == 0): return 1
  elif (x < 1): return 2
  elif (x < 10): return 3
  elif (x < 30): return 4
  elif (x < 70): return 5
  elif (x < 150): return 6
  else: return 7

# df['RF_LeThuy'] = df['RF_LeThuy'].apply(levelOfRainfall)

#### **Split Data to 2 part of year**

In [53]:
dt1 = df[df['Month'].between(1, 8) | df['Month'].between(12, 12)]
dt2 = df[df['Month'].between(9, 11)]

In [54]:
dt1 = dt1.drop('Month', axis=1)
dt2 = dt2.drop('Month', axis=1)

#### **Implement Linear Regression From Sklearn**

In [55]:
from sklearn.linear_model import LinearRegression
model_LR = LinearRegression()

from sklearn.svm import SVC
SVM = SVC()

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
# d = dt1
# n_days = 12
# for i in range(n_days):
#     d[f'Day-{i}'] = d['RF_LeThuy'].shift(-i)
# d = d.dropna()
# d = d.drop(columns=['Day-0'])
# X_first = d.drop('RF_LeThuy', axis=1).astype(float)
# y_first = d['RF_LeThuy'].astype(float)

# X_train, X_test, y_train, y_test = train_test_split(X_first, y_first, train_size=0.8)
# model_LR.fit(X_train, y_train)
# y_pred = model_LR.predict(X_test)
# plt.scatter(y_pred, y_test)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f"Mean Squared Error (MSE): {mse:.9f}")
# print(f"R-squared (R²): {r2:.9f}")

In [57]:
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score

# def scoreOfLinear(df, days):
#   for i in range(days):
#     df[f'Day-{i}'] = df['RF_LeThuy'].shift(-i)
#   df = df.dropna()
#   df = df.drop(columns=['Day-0'])
#   X_first = df.drop('RF_LeThuy', axis=1).astype(float)
#   y_first = df['RF_LeThuy'].astype(float)
#   X_train, X_test, y_train, y_test = train_test_split(X_first, y_first, train_size=0.8, random_state=42)
#   model_LR.fit(X_train, y_train)
#   y_pred = model_LR.predict(X_test)
#   r2 = r2_score(y_test, y_pred)
#   mae = mean_absolute_error(y_test, y_pred)
#   mse = mean_squared_error(y_test, y_pred)
#   print(f'\tScore : r2 = {r2}, mae = {mae}, mse = {mse}')


In [58]:
def scoreOfSVM(d1, days):
  df = d1.copy()
  for j in range(days):
    df[f'Day-{j}'] = df['RF_LeThuy'].shift(-j)
  df = df.dropna()
  df = df.drop(columns=['Day-0'])
  X_first = df.drop('RF_LeThuy', axis=1)
  y_first = df['RF_LeThuy']
  X_train, X_test, y_train, y_test = train_test_split(X_first, y_first, train_size=0.8)
  SVM.fit(X_train, y_train)
  y_pred = SVM.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  accuracy = accuracy_score(y_test, y_pred)
  return accuracy, mse, r2

In [59]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

def scoreOfSVM2(df, days):
  for i in range(days):
    df[f'Day-{i}'] = df['RF_LeThuy'].shift(-i)
  df = df.dropna()
  df = df.drop(columns=['Day-0'])
  X_first = df.drop('RF_LeThuy', axis=1).astype(float)
  y_first = df['RF_LeThuy'].astype(float)

  # Tạo một đối tượng mô hình SVM
  SVM = SVR()
  
  # Định nghĩa lưới tham số để tìm kiếm
  param_grid = {
      'kernel': ['linear', 'rbf', 'poly'],
      'C': [0.1, 1, 10],
      'gamma': [0.01, 0.1, 1]
  }
  
  # Sử dụng Grid Search để tìm kiếm các tham số tối ưu
  grid_search = GridSearchCV(SVM, param_grid, cv=5, scoring='neg_mean_squared_error')
  grid_search.fit(X_first, y_first)
  
  # Lấy ra các tham số tối ưu
  best_params = grid_search.best_params_
  print(f"Best Parameters: {best_params}")
  
  # Tạo lại mô hình SVM với các tham số tối ưu
  best_SVM = SVR(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'])
  
  X_train, X_test, y_train, y_test = train_test_split(X_first, y_first, train_size=0.8)
  
  best_SVM.fit(X_train, y_train)
  y_predSVM = best_SVM.predict(X_test)
  
  mseSVM = mean_squared_error(y_test, y_predSVM)
  r2SVM = r2_score(y_test, y_predSVM)
  
  accuracy = accuracy_score(y_test, y_predSVM)
  print(f'Mean Squared Error (MSE): {mseSVM}')
  print(f'R-squared (R2) Score: {r2SVM}')
  print(f'Accuracy Score: {accuracy}')


In [60]:
accuracyArr = []
mseArr = []
r2Arr = []

# for i in [7, 14, 21, 28, 35]:
#   accuracy, mse, r2 = scoreOfSVM(dt1, i)
#   accuracyArr.append(accuracy)
#   mseArr.append(mse)
#   r2Arr.append(r2)

accuracyArr

[]

In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

def scoreOfLSTM(d1, days):
    df = d1.copy()
    for j in range(1, days + 1):
        df[f'Day-{j}'] = df['RF_LeThuy'].shift(-j)
    
    df = df.dropna()
    X = []
    y = []
    
    for i in range(len(df) - days):
        X.append(df.iloc[i:i + days, 1:].values)
        y.append(df.iloc[i + days, 0])
    
    X = np.array(X)
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)  # Chú ý shuffle=False để đảm bảo sự liên tục trong dữ liệu
    
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(days, df.shape[1] - 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train, y_train, epochs=100, validation_split=0.2)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    # accuracy = accuracy_score(y_test, y_pred)
    return mse, r2

# Sử dụng hàm
mse, r2 = scoreOfLSTM(dt1, 7)  # hoặc days=14
print("Mean Squared Error:", mse)
print("R-squared:", r2)
# print("Accuracy:", accuracy)


Epoch 1/100
242/242 [==============================] - 2s 4ms/step - loss: 32.7350 - val_loss: 9.8761
Epoch 2/100
242/242 [==============================] - 1s 3ms/step - loss: 7.6245 - val_loss: 5.1141
Epoch 3/100
242/242 [==============================] - 1s 3ms/step - loss: 6.1071 - val_loss: 3.9933
Epoch 4/100
242/242 [==============================] - 1s 3ms/step - loss: 6.1386 - val_loss: 2.4852
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: 4.0826 - val_loss: 2.2762
Epoch 6/100
242/242 [==============================] - 1s 3ms/step - loss: 3.5326 - val_loss: 2.0781
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: 3.2423 - val_loss: 1.9313
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: 2.4602 - val_loss: 2.2716
Epoch 9/100
242/242 [==============================] - 1s 3ms/step - loss: 2.6057 - val_loss: 1.4209
Epoch 10/100
242/242 [==============================] - 1s 3ms/step - loss: 1.6206 - val_l